In [68]:
#load the required packages
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
import sklearn.model_selection as ms
import sklearn.metrics as sklm
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np
import numpy.random as nr

%matplotlib inline

In [69]:
#load the dataset using pandas. the varaible dataset will refer to this. 
dataset = pd.read_csv('final dataset')
#check the first 5 rows.
dataset.head()

,customer_id,title,first_name,middle_name,last_name,suffix,address_line_1,address_line_2,city,state_province_name,...,occupation,gender,marital_status,home_owner,number_of_cars_owned,number_of_children_at_home,total_children,yearly_income,average_monthly_expense,bike_buyer
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,...,Professional,M,M,1,0,0,2,137947,4.488636,0
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,...,Professional,M,S,0,1,3,3,101141,4.762174,1
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,...,Professional,M,M,1,1,3,3,91945,4.812184,0
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,...,Professional,F,S,0,1,0,0,86688,3.912023,0
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,...,Professional,F,S,1,4,5,5,92771,4.553877,1


In [70]:
#Check how many variables and cases there are.
print(dataset.shape)

(16471, 25)


In [71]:
#Select the features you want to train your model on 
Features = np.array(dataset[['home_owner', 'number_of_cars_owned',
                             'number_of_children_at_home', 
                             'yearly_income', 
                             'average_monthly_expense']])

#the data under bike_buyer is chosen as the label
Labels = np.array([x for x in dataset['bike_buyer']])

In [72]:
#Randomly sample cases to create independent training and test data
nr.seed(1115)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 8235)
x_train = Features[indx[0],:]
y_train = np.ravel(Labels[indx[0]])
x_test = Features[indx[1],:]
y_test = np.ravel(Labels[indx[1]])

In [73]:
#Scale all the data so that the magnitude of the numbers don't cause the machine to prioritize one data over another
scale = preprocessing.StandardScaler()
scale.fit(x_train)
x_train = scale.transform(x_train)

In [74]:
#Use the Rnadom Forest Classifier and create 50 forests to increase accuracy 
nr.seed(444)
forest = RandomForestClassifier(n_estimators=50)
forest.fit(x_train, y_train)

RandomForestClassifier(n_estimators=50)

In [75]:
#predict the values of the testing data using the classification model that has been created 
x_test = scale.transform(x_test)
scores = forest.predict(x_test)

In [76]:
#compare the prediction with the actual data to check the accuracy
def print_results(labels, scores):
    conf = sklm.confusion_matrix(labels, scores)
    print('                 Confusion matrix')
    print('                 Bike                No Bike')
    print('Bike             %6d' % conf[0,0] + '            %5d' % conf[0,1])
    print('No bike          %6d' % conf[1,0] + '            %5d' % conf[1,1])
    print('')
    print('Accuracy        %0.2f' % sklm.accuracy_score(labels, scores))
    metrics = sklm.precision_recall_fscore_support(labels, scores)
    print(' ')
    print('          Bike    No Bike')
    print('Num case   %0.2f' % metrics[3][0] + '     %0.2f' % metrics[3][1])
    print('Precision   %0.2f' % metrics[0][0] + '      %0.2f' % metrics[0][1])
    print('Recall      %0.2f' % metrics[1][0] + '      %0.2f' % metrics[1][1])
    print('F1          %0.2f' % metrics[2][0] + '      %0.2f' % metrics[2][1])
    
print_results(y_test, scores)    

                 Confusion matrix
                 Bike                No Bike
Bike               4503             1001
No bike            1182             1549

Accuracy        0.73
 
          Bike    No Bike
Num case   5504.00     2731.00
Precision   0.79      0.61
Recall      0.82      0.57
F1          0.80      0.59
